In [1]:
# !mkdir /4TB/popenov/PiCapcha

# !mkdir /4TB/popenov/PiCapcha/dataset
# !mkdir /4TB/popenov/PiCapcha/dataset/train
# !mkdir /4TB/popenov/PiCapcha/dataset/test

# !mkdir /4TB/popenov/PiCapcha/models
# !mkdir /4TB/popenov/PiCapcha/results
# !mkdir /4TB/popenov/PiCapcha/models/MNv2/
# !ls

In [3]:
!nvidia-smi

Sat Dec  8 22:10:57 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.130                Driver Version: 384.130                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:09:00.0 Off |                  N/A |
| 60%   74C    P2   146W / 250W |   6090MiB / 11172MiB |     59%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:0A:00.0 Off |                  N/A |
| 45%   68C    P2    59W / 250W |     10MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [4]:
%env CUDA_VISIBLE_DEVICES=-1

env: CUDA_VISIBLE_DEVICES=-1


In [5]:
SAMPLES_IN_CLASS = 5000
NUM_CLASSES = 345
TEST_PART = 0.15

In [6]:
# !git clone https://github.com/googlecreativelab/quickdraw-dataset

In [7]:
# !cp quickdraw-dataset/categories.txt /4TB/popenov/PiCapcha/dataset/categories.txt

In [8]:
import numpy as np
import os
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from quickdraw import QuickDrawData, QuickDrawDataGroup

In [9]:
DATASET_ROOT = '/4TB/popenov/PiCapcha/dataset'

In [10]:
test_samples_num = int(SAMPLES_IN_CLASS * TEST_PART)
train_samples_num = SAMPLES_IN_CLASS - test_samples_num

In [11]:
# with open(os.path.join(DATASET_ROOT, 'categories.txt')) as f:
#     for name in f.readlines():
#         name = name.strip()
#         data = QuickDrawDataGroup(name, max_drawings=SAMPLES_IN_CLASS, recognized=True)
        
#         i = 0
#         for sample in data.drawings:
#             if i < train_samples:
#                 if not os.path.isdir(os.path.join(DATASET_ROOT, 'train/'+name)):
#                     os.mkdir(os.path.join(DATASET_ROOT, 'train/'+name))
#                 sample.image.save(DATASET_ROOT+'/train/'+name+'/'+str(sample.key_id)+'.png')
#                 i += 1
#             else:
#                 if not os.path.isdir(os.path.join(DATASET_ROOT, 'test/'+name)):
#                         os.mkdir(os.path.join(DATASET_ROOT, 'test/'+name))
#                 sample.image.save(DATASET_ROOT+'/test/'+name+'/'+str(sample.key_id)+'.png')

In [12]:
INPUT_SHAPE = (224,224,3)
LEARNING_RATE = 0.001
NUM_EPOCHS = 100
BATCH_SIZE = 25

In [13]:
import keras
from keras.applications.mobilenet import MobileNet
import tensorflow as tf
from keras.layers import *
from keras.models import *
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.metrics import categorical_accuracy
from keras.applications.imagenet_utils import preprocess_input

Using TensorFlow backend.


In [14]:
# input_tensor = Input(shape=INPUT_SHAPE)
# model = input_tensor
# model = Lambda(preprocess_input, name='preprocessing')(model)
# model = MobileNet(input_shape=(224,224,3), include_top=False, weights='imagenet')(model)
# model = GlobalAveragePooling2D()(model)
# model = Dropout(0.5)(model)
# model = Dense(NUM_CLASSES, activation='softmax')(model)
# model = Model(inputs=input_tensor, outputs=model)
# opt = Adam(lr=LEARNING_RATE, decay=LEARNING_RATE/NUM_EPOCHS)
# model.compile(loss=categorical_crossentropy, optimizer=opt, metrics=[categorical_accuracy])

In [15]:
# import cv2
# TRAIN_PATH = os.path.join(DATASET_ROOT, 'train')
# filepaths = []
# y = dict()
# number2category = dict()
# for dir_number, directory in enumerate(sorted(os.listdir(TRAIN_PATH))):
#     number2category[dir_number] = directory
#     for filename in os.listdir(os.path.join(TRAIN_PATH, directory)):
#         filepath = os.path.join(TRAIN_PATH, directory, filename)
#         filepaths.append(filepath)
#         y[filepath] = dir_number

In [16]:
# class InputGenerator(object):
#     def __init__(self, filepaths, y, batch_size=8, img_size=224, shuffle=True):
#         self.num_classes = NUM_CLASSES
#         def generator():
#             while True:
#                 order = np.arange(len(filepaths))
#                 if shuffle:
#                     np.random.shuffle(order)

#                 n_batches = len(order) // batch_size
#                 for i in range(n_batches):
#                     batch_order = order[i * batch_size: (i + 1) * batch_size]

#                     batch_filepaths = []
#                     for ord in batch_order:
#                         batch_filepaths.append(filepaths[ord])

#                     batch_x = np.zeros((batch_size, img_size, img_size, 3), dtype=np.uint8)
#                     batch_y = np.zeros((batch_size, self.num_classes), dtype=float)
#                     for k, filepath in enumerate(batch_filepaths):
#                         img = cv2.imread(filepath)
#                         img = cv2.resize(img, (img_size, img_size))
#                         batch_x[k] = img[:, :, ::-1]
#                         batch_y[k][y[filepath]] = 1

#                     batch_x = batch_x.astype("float")
#                     yield batch_x, batch_y

#         self.generator = generator()
#         self.steps = len(filepaths) // batch_size

In [17]:
# MODEL_SAVE_PATH = '/4TB/popenov/PiCapcha/models/MNv2/'
# MODEL_DIR_NAME = 'first_try'

# checkpoint = keras.callbacks.ModelCheckpoint(
#             os.path.join(MODEL_SAVE_PATH, MODEL_DIR_NAME)+"_{epoch:02d}.hdf5",
#             monitor='val_loss',
#             verbose=0,
#             save_best_only=False,
#             save_weights_only=False,
#             mode='auto',
#             period=1
# )

# # print(np.unique(list(y.values())))
# # print(filepaths)

# model.fit_generator(InputGenerator(filepaths, y, batch_size=BATCH_SIZE).generator,
#                     epochs=NUM_EPOCHS,
#                     steps_per_epoch=min(1000, len(filepaths) // BATCH_SIZE),
#                     callbacks=[checkpoint]
#                    )

In [38]:
import keras
keras.__version__

'2.2.0'

In [31]:
import sys
sys.version

'3.5.2 (default, Nov 23 2017, 16:37:01) \n[GCC 5.4.0 20160609]'

# Тестирование

In [25]:
from keras.models import load_model
import tensorflow as tf
from keras_applications.imagenet_utils import preprocess_input
from keras_applications.imagenet_utils import _preprocess_symbolic_input
import cv2
import numpy as np

# print(keras.__version__)

MODEL_PATH = '/4TB/popenov/PiCapcha/models/MNv2/first_try_85.hdf5'

model = load_model(MODEL_PATH, custom_objects={'tf':tf, 'relu6':tf.nn.relu6, 
                                              'preprocess_input':preprocess_input,
                                              '_preprocess_symbolic_input':_preprocess_symbolic_input})

2.2.0


In [26]:
def test_on_one_image(model, image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (INPUT_SHAPE[0], INPUT_SHAPE[1]))
    img = img[:, :, ::-1]
    
    model_input = np.expand_dims(img, axis=0)
    model_output = model.predict(model_input)
    
    category_number = np.argmax(model_output)
    
    return category_number

In [28]:
from tqdm import tqdm
TEST_PATH = '/4TB/popenov/PiCapcha/dataset/test/'

answers = []
true_answers = []

number2category = dict()
for dir_number, directory in enumerate(sorted(os.listdir(TEST_PATH))):
    number2category[dir_number] = directory

category2num = {k:v for v,k in number2category.items()}

for directory in os.listdir(TEST_PATH):
    for img_name in tqdm(os.listdir(os.path.join(TEST_PATH, directory))):
        answers.append(test_on_one_image(model, os.path.join(TEST_PATH, directory, img_name)))
        true_answers.append(category2num[directory])
        break
    break

  0%|          | 0/750 [00:00<?, ?it/s]


In [21]:
import pickle
with open('./category_num_to_name.pickle', 'wb') as f:
    pickle.dump(number2category, f)

In [32]:
from sklearn.metrics import accuracy_score

accuracy_score(true_answers, answers)

0.7902531400966184

In [ ]:
img = 

In [83]:
%%timeit
true_answers.append(test_on_one_image(model, os.path.join(TEST_PATH, directory, img_name)))

30.2 ms ± 578 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
